In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/lstm_muse_adj_bn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_fea

In [2]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                            verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')

done


In [3]:
xgb_res = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold5.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# compare
# 1, this fold avg train 0.0358804455727 avg val 0.0393458962779
# 2, this fold avg train 0.0354424333085 avg val 0.0411007665745

# this fold avg train 0.0338484278753 avg val 0.0392710541094
# this fold avg train 0.0322020276233 avg val 0.0408639577693

[0]	train-logloss:0.649444	valid-logloss:0.649341
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[168]	train-logloss:0.077416	valid-logloss:0.083378

0
0.0739732473062 0.0834460328089
[0]	train-logloss:0.645607	valid-logloss:0.645618
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[167]	train-logloss:0.015079	valid-logloss:0.022135

1
0.0133198800429 0.022296489779
[0]	train-logloss:0.647029	valid-logloss:0.646985
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[321]	train-logloss:0.030392	valid-logloss:0.045364

2
0.0283982395222 0.0455821580183
[0]	train-logloss:0.644746	valid-logloss:0.644824
Multiple 

Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[238]	train-logloss:0.012354	valid-logloss:0.021674

1
0.0108033178706 0.0218978960863
[0]	train-logloss:0.647022	valid-logloss:0.647111
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[193]	train-logloss:0.036316	valid-logloss:0.047184

2
0.0337481999052 0.0474111557427
[0]	train-logloss:0.64475	valid-logloss:0.644795
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[188]	train-logloss:0.003149	valid-logloss:0.007972

3
0.00231083676887 0.00809122337957
[0]	train-logloss:0.647881	valid-logloss:0.647959
Multiple eval metrics have been passed: 'valid-logloss' w